# Application of uCS-BME to 3'- and 5'-UTRs of the SARS-CoV-2 

# Required imports

In [7]:
!pip install sklearn seaborn matplotlib tqdm

Defaulting to user installation because normal site-packages is not writeable


In [9]:
import os
from BME import BME as bme
import pandas as pd
import numpy as np
from reweighting import *
from PyRNA import *
from itertools import combinations, permutations
from collections import namedtuple
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil

import matplotlib
from matplotlib.pyplot import hist2d
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams['mathtext.fontset'] = 'cm'
matplotlib.rcParams['font.family'] = 'sans-serif'
matplotlib.rcParams['font.sans-serif'] = ['Tahoma']

# BME with Unassigned Data

In [12]:
# read in experimential 2D peaks 
top = 500
testing = False
if testing: top = 10
IDs = ['3_HVR', '5_UTR', '3_UTR']
IDs = ['5_UTR', '3_UTR']
N_errors = [1.86, 1.87, 1.88, 1.89, 1.90, 1.91, 1.92]
H_errors = [0.36, 0.37, 0.38, 0.39, 0.40, 0.41, 0.42]
for ID in IDs:
    # read in experimental 2D peaks
    expcs_paired = read_peaks('SARS-CoV-2/%s/iminos_experimental.csv'%(ID))

    # read in simulated chemical shifts and convert to 2D peaks
    simcs = read_computed_cs('SARS-CoV-2/%s/iminos_simulated.csv'%(ID), names = ['model', 'resid', 'resname', 'nucleus', 'simcs', 'id'])
    simcs = simcs[simcs.model < top+1]
    simcs = pd.concat([simcs[simcs.resname.isin(['GUA']) & simcs.nucleus.isin(['N1', 'H1'])], simcs[simcs.resname.isin(['URA']) & simcs.nucleus.isin(['N3', 'H3'])]])
    simcs_paired = create_paired_data_simulated(simcs, csname = "simcs", debug = False, pairing = {"H1":"N1", "H3":"N3"})

    # generate histograms
    for error_index_N in range(0, len(N_errors)):
        for error_index_H in range(0, len(H_errors)):
            expcs_hist, simcs_hist = create_histogram_data(expcs_paired, simcs_paired, xrange = [expcs_paired['F1'].min(),expcs_paired['F1'].max()], xgrid_size = N_errors[error_index_N], yrange = [expcs_paired['F2'].min(), expcs_paired['F2'].max()], ygrid_size = H_errors[error_index_H], x_name = "F1", y_name = "F2")

            # get priors
            energy = pd.read_csv("SARS-CoV-2/%s/energies.csv"%ID, delim_whitespace = True, header = None)
            energy.columns = ['model', 'E']

            # only retain data for the top N conformers
            sub_simcs_hist = {}
            sub_E = []
            for i in range(1, top+1):
                sub_simcs_hist[i] =  simcs_hist[i]  
                sub_E.append(energy.E[i-1])

            w0 = boltzmann_weight(np.array(sub_E), T = 310)
            w0 = w0/np.sum(w0)


            # write files for BME
            bme_exp = "data/bme_exp.dat"
            bme_sim = "data/bme_sim.dat"
            expcs_bme, simcs_bme = write_BME_chemical_shifts_unassigned(expcs_hist, simcs_hist, output_name_exp = "data/bme_exp.dat", output_name_sim = "data/bme_sim.dat", error = 1)
            bmea = bme.Reweight("test")     
            bmea.load(bme_exp , bme_sim)
            
            theta, avg_phi = bmea.theta_scan("data/tmp/%s_%s_%s/"%(ID, error_index_N, error_index_H), [i for i in np.linspace(1,100, 200)])
            optimal_weights, chi2_before, chi2_after, srel = find_weights(bme_exp , bme_sim, theta = theta)

            # collect data
            n_conformers = len(optimal_weights)
            weights = pd.DataFrame({"model": [i for i in range(1, n_conformers+1)], "energy": sub_E, "w": optimal_weights, "errors": [ np.sum(np.abs(expcs_hist - simcs_hist[i])) for i in range(1, 1+n_conformers)]})
            weights = weights.round(5)
            weights.sort_values(by="w")
            weights.plot(x = "w", y = "errors", kind = "scatter")
            plt.savefig("figure_histogram_errors_%s_best_%s_%s.pdf"%(ID, error_index_N, error_index_H))

            # plot CS-BME
            xlim = (expcs_paired['F1'].min(), expcs_paired['F1'].max())
            ylim = (expcs_paired['F2'].min(), expcs_paired['F2'].max())
            xgrid_size = N_errors[error_index_N]
            ygrid_size = H_errors[error_index_H]
            xbins = int((xlim[1] - xlim[0])/xgrid_size)
            ybins = int((ylim[1] - ylim[0])/ygrid_size)
            best = weights.model[np.argmax(weights.w.values)]
            diff = np.sum(np.abs(expcs_hist - simcs_hist[best]))
            title = "highest weighted [%s]: E = %4.1f histogram error = %i"%(best, energy.E[best-1], diff)
            plt.figure()  # create a new figure
            ax = plt.subplot(222) # create the left-side subplot
            expcs_paired.plot(ax=ax, kind = "scatter", x="F1", y="F2", c = "red", xlim = xlim, ylim = ylim, title = title)
            simcs_paired[best].plot(ax=ax, kind = "scatter", x="F1", y="F2", c = "blue", xlim = xlim, ylim = ylim)
            ax.set_xticks(np.geomspace(xlim[0],xlim[1],xbins), minor=True )
            ax.set_yticks(np.geomspace(ylim[0],ylim[1],ybins), minor=True )
            ax.grid('on', which='minor', axis='x' )
            ax.grid('on', which='minor', axis='y' )
            ax.tick_params(which='major', length=0, axis='x')
            ax.tick_params(which='major', length=0, axis='y')
            plt.savefig("figure_%s_best_%s_%s.pdf"%(ID, error_index_N, error_index_H))

            # copy CT files
            oldfile = "SARS-CoV-2/%s/user_all_%s.ct"%(ID, best)
            newfile = "SARS-CoV-2/Best/%s_CSBME_%s_%s.ct"%(ID, error_index_N, error_index_H)
            shutil.copyfile(oldfile, newfile)

            # plot Best
            best = 1
            diff = np.sum(np.abs(expcs_hist - simcs_hist[best]))
            title = "lowest energy[1]: E = %4.1f histogram error = %i"%(energy.E[best-1], diff)
            plt.figure()  # create a new figure
            ax = plt.subplot(222) # create the left-side subplot
            expcs_paired.plot(ax=ax, kind = "scatter", x="F1", y="F2", c = "red", xlim = xlim, ylim = ylim, title = title)
            simcs_paired[best].plot(ax=ax, kind = "scatter", x="F1", y="F2", c = "blue", xlim = xlim, ylim = ylim)
            ax.set_xticks(np.geomspace(xlim[0],xlim[1],xbins), minor=True )
            ax.set_yticks(np.geomspace(ylim[0],ylim[1],ybins), minor=True )
            ax.grid('on', which='minor', axis='x' )
            ax.grid('on', which='minor', axis='y' )
            ax.tick_params(which='major', length=0, axis='x')
            ax.tick_params(which='major', length=0, axis='y')
            plt.savefig("figure_%s_lowest_%s_%s.pdf"%(ID, error_index_N, error_index_H))
            plt.close('all')
            if testing: break
        if testing: break
    if testing: break

    # copy CT files
    oldfile = "SARS-CoV-2/%s/user_all_%s.ct"%(ID, 1)
    newfile = "SARS-CoV-2/Best/%s_Fold.ct"%(ID)
    shutil.copyfile(oldfile, newfile)

100%|██████████| 10/10 [00:08<00:00,  1.20it/s]

Performing 5-fold cross validation for 200 theta values
Output files are written to theta_scan_data/tmp/5_UTR_0_0/


[============================================================] 100.0% ...

Optimal theta: 100.00
Validation error reduction 1.002
Training error reduction 0.999
Fraction of effective frames  0.998


In [3]:
N_grid, H_grid = [],[]
for ID in IDs:
    for i,N in enumerate(N_errors):
        for j, H in enumerate(H_errors):
            print(ID, i, j, N, H, "TPR", "PPV")
    

5_UTR 0 0 1.86 0.36 TPR PPV
5_UTR 0 1 1.86 0.37 TPR PPV
5_UTR 0 2 1.86 0.38 TPR PPV
5_UTR 0 3 1.86 0.39 TPR PPV
5_UTR 0 4 1.86 0.4 TPR PPV
5_UTR 0 5 1.86 0.41 TPR PPV
5_UTR 0 6 1.86 0.42 TPR PPV
5_UTR 1 0 1.87 0.36 TPR PPV
5_UTR 1 1 1.87 0.37 TPR PPV
5_UTR 1 2 1.87 0.38 TPR PPV
5_UTR 1 3 1.87 0.39 TPR PPV
5_UTR 1 4 1.87 0.4 TPR PPV
5_UTR 1 5 1.87 0.41 TPR PPV
5_UTR 1 6 1.87 0.42 TPR PPV
5_UTR 2 0 1.88 0.36 TPR PPV
5_UTR 2 1 1.88 0.37 TPR PPV
5_UTR 2 2 1.88 0.38 TPR PPV
5_UTR 2 3 1.88 0.39 TPR PPV
5_UTR 2 4 1.88 0.4 TPR PPV
5_UTR 2 5 1.88 0.41 TPR PPV
5_UTR 2 6 1.88 0.42 TPR PPV
5_UTR 3 0 1.89 0.36 TPR PPV
5_UTR 3 1 1.89 0.37 TPR PPV
5_UTR 3 2 1.89 0.38 TPR PPV
5_UTR 3 3 1.89 0.39 TPR PPV
5_UTR 3 4 1.89 0.4 TPR PPV
5_UTR 3 5 1.89 0.41 TPR PPV
5_UTR 3 6 1.89 0.42 TPR PPV
5_UTR 4 0 1.9 0.36 TPR PPV
5_UTR 4 1 1.9 0.37 TPR PPV
5_UTR 4 2 1.9 0.38 TPR PPV
5_UTR 4 3 1.9 0.39 TPR PPV
5_UTR 4 4 1.9 0.4 TPR PPV
5_UTR 4 5 1.9 0.41 TPR PPV
5_UTR 4 6 1.9 0.42 TPR PPV
5_UTR 5 0 1.91 0.36 TPR PPV
5_UT